# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1. A hypothesis test on pearson correlation would work well here, permuting one of the variables (b/w) and seeing how it relates to the other (call).  CLT does apply here because this should be a binomial distribution, which CLT would typically apply to.

2.Null Hypothesis = There is no correlation between race and the likelyhood of getting a call for an interview.
Alternate Hypothesis = There is in fact a correlation between race and receiving a call, the assumed outcome is that
a person with black race will be less likely to get called,

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [12]:
#this function will create multiple bootstraps from an array
def draw_bs_reps(data, func, size=1):


    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates


#Here we are testing the null hypothesis, that race has no effect on call rate.  We do this by shifting our w and b array so that
#the means are the same as the combined mean of the two arrays.  We will create a large number of bootstraps assuming all races
#get the same number of calls and see how many cases produce more extreme correlations than the one present in our data.

#w = w.call.values
#b = b.call.values

mean_diff = np.mean(w)-np.mean(b)
combined_mean = np.mean(np.concatenate((w,b)))
w_shifted = w - np.mean(w) + combined_mean
b_shifted = b - np.mean(b) + combined_mean

bs_replicates_w = draw_bs_reps(w_shifted,np.mean,10000)
bs_replicates_b = draw_bs_reps(b_shifted,np.mean,10000)

bs_diff_replicates = bs_replicates_w - bs_replicates_b

p = np.sum(bs_diff_replicates >= mean_diff) / len(bs_diff_replicates)
print(p)

#A very low p value indicates (or a value of 0) indicates that the hypothesis is false! It is likely that race does in fact
#effect the likelyhood of getting called in for an interview.

print(np.percentile(bs_diff_replicates,[2.5,97.5]))

0.0
[-0.0151951   0.01519505]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>